In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

sns.set()
from datetime import datetime


In [6]:
path = r"C:\Users\crati\Desktop\my_pyton_project"

campaign = pd.read_csv(path + r"\dataset_fashion_store_campaigns.csv")
channel = pd.read_csv(path + r"\dataset_fashion_store_channels.csv")
customer = pd.read_csv(path + r"\dataset_fashion_store_customers.csv")
products = pd.read_csv(path + r"\dataset_fashion_store_products.csv")
sales = pd.read_csv(path + r"\dataset_fashion_store_sales.csv")
salesitem = pd.read_csv(path + r"\dataset_fashion_store_salesitems.csv")
stock = pd.read_csv(path + r"\dataset_fashion_store_stock.csv")

In [7]:
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
    )
    return df

In [8]:
campaign["start_date"] = pd.to_datetime(campaign["start_date"], errors="coerce")
campaign["end_date"] = pd.to_datetime(campaign["end_date"], errors="coerce")

customer["signup_date"] = pd.to_datetime(customer["signup_date"], errors="coerce")

sales["sale_date"] = pd.to_datetime(sales["sale_date"], errors="coerce")

salesitem["sale_date"] = pd.to_datetime(salesitem["sale_date"], errors="coerce")

In [9]:
salesitem["sale_year"] = salesitem["sale_date"].dt.year
salesitem["sale_month"] = salesitem["sale_date"].dt.month
salesitem["sale_weekday"] = salesitem["sale_date"].dt.weekday
salesitem["is_weekend"] = salesitem["sale_weekday"].isin([5,6]).astype(int)

In [10]:
products = products.drop(columns=['size'])

In [11]:
salesitem["calculated_total"] = (
    salesitem["unit_price"] * salesitem["quantity"]
)

salesitem["price_inconsistency"] = (
    salesitem["calculated_total"] != salesitem["item_total"]
).astype(int)

In [12]:
salesitem.loc[
    salesitem["price_inconsistency"] == 1,
    "item total"
] = salesitem["calculated_total"]

In [13]:
salesitem = salesitem[salesitem["quantity"] > 0]
sales = sales[sales["total_amount"] > 0]
products = products[products["catalog_price"] >= products["cost_price"]]

In [14]:
products['color'] = products["color"].fillna("Unknown")
customer["age_range"] = customer["age_range"].fillna("Unknown")

In [15]:
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
    )
    return df

campaign = clean_columns(campaign)
channel = clean_columns(channel)
customer = clean_columns(customer)
products = clean_columns(products)
sales = clean_columns(sales)
salesitem = clean_columns(salesitem)
stock = clean_columns(stock)

In [16]:
fact = salesitem.merge(
    sales,
    on=["sale_id", "sale_date", "channel"],
    how="left"
)

In [17]:
fact = fact.merge(customer, on="customer_id", how="left")

In [18]:
fact = fact.merge(products, on="product_id", how="left")

In [21]:
fact = fact.merge(campaign, left_on="channel_campaigns", right_on="channel", how="left")

In [ ]:
cols_to_drop = [c for c in fact.columns if c.endswith('_y')]
fact = fact.drop(columns=cols_to_drop)
fact.columns = [c.replace('_x', '') for c in fact.columns]
print(fact.columns)

In [22]:
fact["price_margin"] = (
    fact["catalog_price"] - fact["cost_price"]
)

In [30]:
fact["campaign_active"] = (
    (fact["sale_date"] >= fact["start_date"]) &
    (fact["sale_date"] <= fact["end_date"])
).astype(int)

In [31]:
fact["discount_ratio"] = (
    fact["discount_applied"] / fact["original_price"]
).fillna(0)

In [32]:
fact["target_revenue"] = fact["item_total"]

In [33]:
threshold = fact["item_total"].quantile(0.75)
fact["high_value_sale"] = (
    fact["item_total"] >= threshold
).astype(int)

In [ ]:
fact.shape, fact.isna().sum().sort_values(ascending=False).head(10)